In [12]:
# 1. Imports
import os
import time
import requests
from PIL import Image
from io import BytesIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:

# 2. Setup headless Chrome
print("Setting up Chrome...")
options = Options()
#options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # 3. Load page
    print("Loading page...")
    url = "https://qiandao.com/island/catalog?id=300569&navigationName=%E5%9B%BE%E9%89%B4&tabName=%E8%B0%B7%E5%AD%90%E7%B3%BB%E5%88%97&title=%E8%B0%B7%E5%AD%90"
    driver.get(url)

    # 4. Wait for tab container
    print("Waiting for tab container to load...")
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "du-tabs__content")))
    print("Tab container loaded.")

    # 5. Try finding the '通行认证' tab
    print("Looking for tabs...")
    tab_elements = driver.find_elements(By.CSS_SELECTOR, ".du-tab-item")
    print(f"Found {len(tab_elements)} tab elements.")

    target_tab = None
    for i, tab in enumerate(tab_elements):
        print(f"Tab {i}: {tab.text.strip()}")
        if "通行认证" in tab.text:
            target_tab = tab
            print(f"Found target tab: {tab.text.strip()}")
            break

    if not target_tab:
        raise Exception("❌ Tab with text '通行认证' not found.")

    print("Clicking on '通行认证' tab...")
    driver.execute_script("arguments[0].click();", target_tab)
    time.sleep(3)

    # 6. Scroll to bottom
    print("Scrolling to load all images...")
    scroll_pause = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("Finished scrolling.")

    # 7. Create folder
    output_folder = "txz_imgs"
    os.makedirs(output_folder, exist_ok=True)

    # 8. Extract image info
    print("Extracting image elements...")
    image_elements = driver.execute_script("""
    return Array.from(document.querySelectorAll('img')).map(img => {
      return {
        src: img.getAttribute('src') || img.getAttribute('data-src'),
        alt: img.getAttribute('alt') || 'no_alt'
      };
    });
    """)
    print(f"✅ Found {len(image_elements)} images.")

    # 9. Sanitize and download
    def sanitize_filename(name):
        return "".join(c for c in name if c.isalnum() or c in (' ', '_', '-')).rstrip()

    for img in image_elements:
        src = img['src']
        alt = img['alt']
        filename = sanitize_filename(alt)[:100]
        if not src or not src.startswith("http"):
            continue
        try:
            response = requests.get(src, timeout=10)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            path = os.path.join(output_folder, f"{filename}.jpg")
            image.save(path, "JPEG")
            print(f"Saved: {path}")
        except Exception as e:
            print(f"Failed to save {filename}: {e}")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    driver.quit()
    print("Browser closed.")


Setting up Chrome...
Loading page...
Waiting for tab container to load...
Tab container loaded.
Looking for tabs...
Found 4 tab elements.
Tab 0: 角色
Tab 1: 谷子系列
Tab 2: 通行认证
Found target tab: 通行认证
Clicking on '通行认证' tab...
Scrolling to load all images...
Finished scrolling.
Extracting image elements...
✅ Found 769 images.
Saved: txz_imgs\no_alt.jpg
Saved: txz_imgs\水灯心-通行认证470.jpg
Saved: txz_imgs\钼铅精英二VER-通行认证470.jpg
Saved: txz_imgs\水灯心精英二VER-通行认证470.jpg
Saved: txz_imgs\能天使-嘉年华2022通行认证sp.jpg
Saved: txz_imgs\Mon3tr精英二VER-通行认证470.jpg
Saved: txz_imgs\CONFESS-47-通行认证480.jpg
Saved: txz_imgs\石棉精英二VER-通行认证470.jpg
Saved: txz_imgs\乌尔比安精英二VER-通行认证400.jpg
Saved: txz_imgs\夕精英二VER-通行认证210.jpg
Saved: txz_imgs\令精英二VER-通行认证210.jpg
Saved: txz_imgs\黍精英二VER-通行认证360.jpg
Saved: txz_imgs\菲莱精英二VER-通行认证440.jpg
Saved: txz_imgs\艾丽妮精英二VER-通行认证410.jpg
Saved: txz_imgs\蕾缪安精英二VER-通行认证480.jpg
Saved: txz_imgs\维什戴尔-通行认证390.jpg
Saved: txz_imgs\濯尘芙蓉-通行认证130.jpg
Saved: txz_imgs\归溟幽灵鲨-通行认证400.jpg
Saved: txz_imgs\缄默德克萨斯精英二VER-

In [2]:
import os
import shutil

# Input and output folders
src_folder = "txz_imgs"
dst_folder = "txz_imgs_1"
os.makedirs(dst_folder, exist_ok=True)

# Loop over .jpg files
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    name = filename[:-4]  # Strip ".jpg"
    parts = name.split("-")

    if len(parts) < 2:
        continue  # Doesn't match expected pattern

    a = "-".join(parts[:-1])
    b = parts[-1]

    if "精英二" in a:
        continue
    if "sp" in b.lower():
        continue
    
    # Extract numeric part from the last segment
    numeric_part = ''.join(filter(str.isdigit, b))
    if numeric_part:
        a = f"{a}_{numeric_part}"

    # Save as "a.jpg" in destination
    src_path = os.path.join(src_folder, filename)
    dst_path = os.path.join(dst_folder, f"{a}.jpg")

    try:
        shutil.copyfile(src_path, dst_path)
        print(f"Copied: {filename} → {a}.jpg")
    except Exception as e:
        print(f"Error copying {filename}: {e}")


Copied: 12F-通行认证20.jpg → 12F_20.jpg
Copied: Castle-3-通行认证90.jpg → Castle-3_90.jpg
Copied: CONFESS-47-通行认证480.jpg → CONFESS-47_480.jpg
Copied: FRISTON-3-通行认证270.jpg → FRISTON-3_270.jpg
Copied: Lancet-2-通行认证70.jpg → Lancet-2_70.jpg
Copied: Mon3tr-通行认证470.jpg → Mon3tr_470.jpg
Copied: PhonoR-0-通行认证480.jpg → PhonoR-0_480.jpg
Copied: THRM-EX-象限解构者演职认证挂件.jpg → THRM-EX.jpg
Copied: THRM-EX-通行认证20.jpg → THRM-EX_20.jpg
Copied: U-Official-通行认证250.jpg → U-Official_250.jpg
Copied: W-通行认证10.jpg → W_10.jpg
Copied: W通行认证-愚夜密函角色套组.jpg → W通行认证.jpg
Copied: 万顷-通行认证360.jpg → 万顷_360.jpg
Copied: 临光-通行认证90.jpg → 临光_90.jpg
Copied: 乌尔比安-通行认证400.jpg → 乌尔比安_400.jpg
Copied: 乌有-通行认证210.jpg → 乌有_210.jpg
Copied: 云迹-通行认证440.jpg → 云迹_440.jpg
Copied: 亚叶-通行认证390.jpg → 亚叶_390.jpg
Copied: 仇白-通行认证240.jpg → 仇白_240.jpg
Copied: 令-通行认证210.jpg → 令_210.jpg
Copied: 伊内丝-通行认证250.jpg → 伊内丝_250.jpg
Copied: 伊桑-通行认证20.jpg → 伊桑_20.jpg
Copied: 伊芙利特-通行认证140.jpg → 伊芙利特_140.jpg
Copied: 休谟斯-通行认证250.jpg → 休谟斯_250.jpg
Copied: 伺夜-通行认证190.jpg → 伺夜

In [6]:
import os
import cv2
import numpy as np


def correct_rotation(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # 霍夫直线变换
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)
    if lines is not None:
        # 取所有直线的角度，统计出现最多的角度
        angles = []
        for line in lines:
            rho, theta = line[0]
            angle = (theta * 180 / np.pi) - 90  # 转为角度，0为竖直
            # 只考虑接近水平或竖直的线
            if -45 < angle < 45:
                angles.append(angle)
        if len(angles) > 0:
            median_angle = np.median(angles)
            # 旋转校正
            (h, w) = img.shape[:2]
            M = cv2.getRotationMatrix2D((w // 2, h // 2), median_angle, 1.0)
            rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, borderValue=(255,255,255))
            return rotated
    # 如果没有检测到直线，返回原图
    return img


def extract_acrylic_board_by_projection(image_path, output_path):
    img = cv2.imread(image_path)
    img = correct_rotation(img)  # 自动旋转校正
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # 调整投影阈值
    threshold = 30

    # 竖直方向投影
    vertical_proj = np.sum(blurred, axis=1).astype(np.float32)
    # 对投影做一阶差分，找突变点
    diff = np.diff(vertical_proj)
    # 找到第一个diff<0的点和最后一个diff>0的点（分别对应上下边界）
    top = np.argmax(diff < -threshold)
    bottom = len(diff) - np.argmax(diff[::-1] > threshold) - 1

    # 水平方向投影
    horizontal_proj = np.sum(blurred, axis=0).astype(np.float32)
    diff = np.diff(horizontal_proj)
    # 找到第一个diff<0的点和最后一个diff>0的点（分别对应上下边界）
    left = np.argmax(diff < -threshold)
    right = len(diff) - np.argmax(diff[::-1] > threshold) - 1

    # 按比例裁剪亚克力板
    board_height = bottom - top
    acrylic_top = int(top + 0.41 * board_height)
    acrylic_bottom = bottom
    board = img[acrylic_top:acrylic_bottom, left:right]

    # 插值到固定大小
    board = cv2.resize(board, (554, 1000))
    print(board.shape[1] / board.shape[0])

    cv2.imwrite(output_path, board)
    print(f"已保存到 {output_path}")



In [11]:
# for image in txz_imgs, extract_acrylic_board_by_projection
src_folder = "txz_imgs_utf8"
dst_folder = "txz_imgs_extract"
os.makedirs(dst_folder, exist_ok=True)
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    name = filename[:-4]  # Strip ".jpg"
    src_path = os.path.join(src_folder, filename)
    dst_path = os.path.join(dst_folder, f"{name}.jpg")

    try:
        extract_acrylic_board_by_projection(src_path, dst_path)
    except Exception as e:
        print(f"Error processing {filename}: {e}")

0.554
已保存到 txz_imgs_extract\3132462e6a7067.jpg
0.554
已保存到 txz_imgs_extract\434f4e464553532d34372e6a7067.jpg
0.554
已保存到 txz_imgs_extract\436173746c652d332e6a7067.jpg
0.554
已保存到 txz_imgs_extract\46524953544f4e2d332e6a7067.jpg
0.554
已保存到 txz_imgs_extract\4c616e6365742d322e6a7067.jpg
0.554
已保存到 txz_imgs_extract\4d6f6e3374722e6a7067.jpg
0.554
已保存到 txz_imgs_extract\50686f6e6f522d302e6a7067.jpg
0.554
已保存到 txz_imgs_extract\5448524d2d45582e6a7067.jpg
0.554
已保存到 txz_imgs_extract\552d4f6666696369616c2e6a7067.jpg
0.554
已保存到 txz_imgs_extract\572e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4b887e9a1b72e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4b8b4e585892e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4b98ce5b094e6af94e5ae892e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4b98ce69c892e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4ba91e8bfb92e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4ba9ae58fb62e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4bb87e799bd2e6a7067.jpg
0.554
已保存到 txz_imgs_extract\e4bba42e6a7067.jpg
0.554
已保存到 txz_imgs_

In [25]:
import os
from PIL import Image, ImageChops

# Input and output folders
src_folder = "txz_imgs_1"
dst_folder = "txz_imgs_crop"
os.makedirs(dst_folder, exist_ok=True)

# Target size
target_size = (200, 400)

def crop_with_xywh(image, x, y, w, h):
    return image.crop((x, y, x + w, y + h))

# Process each image in the source folder
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    src_path = os.path.join(src_folder, filename)
    dst_path = os.path.join(dst_folder, filename)

    try:
        # Open the image
        with Image.open(src_path) as img:
            # crop the image according to filenames
            name = filename[:-4]
            if name.endswith("_70"):
                img = crop_with_xywh(img, 128, 208, 109, 218)
            elif name.endswith("_20"):
                img = crop_with_xywh(img, 127, 211, 105, 210)
            elif name.endswith("_360"):
                img = crop_with_xywh(img, 118, 205, 126, 252)
            elif name.endswith("_410"):
                img = crop_with_xywh(img, 117, 204, 126, 252)
            elif name.endswith("_440") or name.endswith("_450"):
                img = crop_with_xywh(img, 113, 207, 130, 250)
            else:
                img = crop_with_xywh(img, 113, 207, 132, 264)
            # Resize the image
            img = img.resize(target_size, Image.ANTIALIAS)

            # Save the processed image
            img.save(dst_path, "JPEG")
            print(f"Processed and saved: {dst_path}")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

C:\Users\hexaa\AppData\Local\Temp\ipykernel_15584\2400786029.py:41: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize(target_size, Image.ANTIALIAS)


Processed and saved: txz_imgs_crop\12F_20.jpg
Processed and saved: txz_imgs_crop\Castle-3_90.jpg
Processed and saved: txz_imgs_crop\CONFESS-47_480.jpg
Processed and saved: txz_imgs_crop\FRISTON-3_270.jpg
Processed and saved: txz_imgs_crop\Lancet-2_70.jpg
Processed and saved: txz_imgs_crop\Mon3tr_470.jpg
Processed and saved: txz_imgs_crop\PhonoR-0_480.jpg
Processed and saved: txz_imgs_crop\THRM-EX_20.jpg
Processed and saved: txz_imgs_crop\U-Official_250.jpg
Processed and saved: txz_imgs_crop\W_10.jpg
Processed and saved: txz_imgs_crop\万顷_360.jpg
Processed and saved: txz_imgs_crop\临光_90.jpg
Processed and saved: txz_imgs_crop\乌尔比安_400.jpg
Processed and saved: txz_imgs_crop\乌有_210.jpg
Processed and saved: txz_imgs_crop\云迹_440.jpg
Processed and saved: txz_imgs_crop\亚叶_390.jpg
Processed and saved: txz_imgs_crop\仇白_240.jpg
Processed and saved: txz_imgs_crop\令_210.jpg
Processed and saved: txz_imgs_crop\伊内丝_250.jpg
Processed and saved: txz_imgs_crop\伊桑_20.jpg
Processed and saved: txz_imgs_crop\

In [26]:
# for images end in 460, make the #361a16 color close to white
import os
from PIL import Image

# Input and output folders
src_folder = "txz_imgs_crop"
dst_folder = "txz_imgs_corrected"
os.makedirs(dst_folder, exist_ok=True)

# Define the target color range (dark brown) and replacement color (white)
target_color = (54, 26, 22)  # RGB for #361a16
tolerance = 20  # Adjust tolerance for near colors
replacement_color = (255, 255, 255)  # White

def is_within_tolerance(color, target, tolerance):
    return all(abs(color[i] - target[i]) <= tolerance for i in range(3))

# Process each image in the source folder
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    if not filename.endswith("_460.jpg"):
        continue

    src_path = os.path.join(src_folder, filename)
    dst_path = os.path.join(dst_folder, filename)

    try:
        # Open the image
        with Image.open(src_path) as img:
            img = img.convert("RGB")  # Ensure image is in RGB mode
            pixels = img.load()

            # Replace target color and near colors with white
            for y in range(img.height):
                for x in range(img.width):
                    if is_within_tolerance(pixels[x, y], target_color, tolerance):
                        pixels[x, y] = replacement_color

            # Save the corrected image
            img.save(dst_path, "JPEG")
            print(f"Corrected and saved: {dst_path}")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

Corrected and saved: txz_imgs_corrected\宴_460.jpg
Corrected and saved: txz_imgs_corrected\引星棘刺_460.jpg
Corrected and saved: txz_imgs_corrected\清道夫_460.jpg
Corrected and saved: txz_imgs_corrected\特克诺_460.jpg
Corrected and saved: txz_imgs_corrected\瑰盐_460.jpg
Corrected and saved: txz_imgs_corrected\诺威尔_460.jpg
Corrected and saved: txz_imgs_corrected\隐德来希_460.jpg


In [28]:
import os
from PIL import Image

# Input and output folders
src_folder = "txz_imgs_crop"
dst_folder = "txz_pixelated"
os.makedirs(dst_folder, exist_ok=True)

# Pixelation factor (higher value = more pixelated)
pixelation_factor = 20

# Process each image in the source folder
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    src_path = os.path.join(src_folder, filename)
    dst_path = os.path.join(dst_folder, filename)

    try:
        # Open the image
        with Image.open(src_path) as img:
            img = img.convert("RGB")  # Ensure RGB mode
            original_size = img.size

            # Resize to smaller size
            small_size = (original_size[0] // pixelation_factor, original_size[1] // pixelation_factor)
            img_small = img.resize(small_size, Image.NEAREST)

            # Create a new image with the same size as the original
            pixelated_img = Image.new("RGB", original_size)

            # Scale up each pixel block to the original size
            for y in range(small_size[1]):
                for x in range(small_size[0]):
                    # Get the color of the small pixel
                    color = img_small.getpixel((x, y))
                    # Calculate the position of the block in the original image
                    x_start = x * pixelation_factor
                    y_start = y * pixelation_factor
                    x_end = min(x_start + pixelation_factor, original_size[0])
                    y_end = min(y_start + pixelation_factor, original_size[1])
                    # Fill the block with the same color
                    for yy in range(y_start, y_end):
                        for xx in range(x_start, x_end):
                            pixelated_img.putpixel((xx, yy), color)

            # Save the pixelated image
            pixelated_img.save(dst_path, "JPEG")
            print(f"Pixelated and saved: {dst_path}")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

Pixelated and saved: txz_pixelated\12F_20.jpg
Pixelated and saved: txz_pixelated\Castle-3_90.jpg
Pixelated and saved: txz_pixelated\CONFESS-47_480.jpg
Pixelated and saved: txz_pixelated\FRISTON-3_270.jpg
Pixelated and saved: txz_pixelated\Lancet-2_70.jpg
Pixelated and saved: txz_pixelated\Mon3tr_470.jpg
Pixelated and saved: txz_pixelated\PhonoR-0_480.jpg
Pixelated and saved: txz_pixelated\THRM-EX_20.jpg
Pixelated and saved: txz_pixelated\U-Official_250.jpg
Pixelated and saved: txz_pixelated\W_10.jpg
Pixelated and saved: txz_pixelated\万顷_360.jpg
Pixelated and saved: txz_pixelated\临光_90.jpg
Pixelated and saved: txz_pixelated\乌尔比安_400.jpg
Pixelated and saved: txz_pixelated\乌有_210.jpg
Pixelated and saved: txz_pixelated\云迹_440.jpg
Pixelated and saved: txz_pixelated\亚叶_390.jpg
Pixelated and saved: txz_pixelated\仇白_240.jpg
Pixelated and saved: txz_pixelated\令_210.jpg
Pixelated and saved: txz_pixelated\伊内丝_250.jpg
Pixelated and saved: txz_pixelated\伊桑_20.jpg
Pixelated and saved: txz_pixelated\

In [1]:
import os
import json
from PIL import Image

# Input folder and output JSON file
src_folder = "txz_imgs_crop"
output_json = "operators.json"

# Area to calculate average color (x, y, w, h)
crop_area = (40, 240, 160, 160)

# Dictionary to store results
results = {}

def chinese_to_unicode_key(name: str) -> str:
    return ''.join(f'u{ord(char):04x}' for char in name)

# Process each image in the source folder
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    # Extract name and index from the filename
    name, index = filename[:-4].rsplit("_", 1)

    src_path = os.path.join(src_folder, filename)

    try:
        # Open the image
        with Image.open(src_path) as img:
            img = img.convert("RGB")  # Ensure RGB mode

            # Crop the specified area
            cropped = img.crop((crop_area[0], crop_area[1], crop_area[0] + crop_area[2], crop_area[1] + crop_area[3]))

            # Calculate the average color
            pixels = list(cropped.getdata())
            avg_color = tuple(sum(c[i] for c in pixels) // len(pixels) for i in range(3))
            hex_color = "#{:02x}{:02x}{:02x}".format(*avg_color)
            
            # convert name to utf-8 string
            unicode_name = chinese_to_unicode_key(name)

            # Add to results
            results[unicode_name] = {
                "name": name,
                "unicode": unicode_name,
                "index": int(index),
                "hex": hex_color
            }

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Save results to JSON
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Average colors saved to {output_json}")

Average colors saved to config.json


In [2]:
import os
import shutil

# Input and output folders
src_folder = "txz_pixelated"
dst_folder = "txz"
os.makedirs(dst_folder, exist_ok=True)

# Function to convert name to Unicode string
def chinese_to_unicode_key(name: str) -> str:
    return ''.join(f'u{ord(char):04x}' for char in name)

# Process each image in the source folder
for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue

    # Extract name without extension
    name, index = filename[:-4].rsplit("_", 1)

    # Convert name to Unicode
    unicode_name = chinese_to_unicode_key(name)

    # Define source and destination paths
    src_path = os.path.join(src_folder, filename)
    dst_path = os.path.join(dst_folder, f"{unicode_name}.jpg")

    try:
        # Copy the file with the new name
        shutil.copyfile(src_path, dst_path)
        print(f"Copied and saved: {dst_path}")
    except Exception as e:
        print(f"Error processing {filename}: {e}")

Copied and saved: txz\u0031u0032u0046.jpg
Copied and saved: txz\u0043u0061u0073u0074u006cu0065u002du0033.jpg
Copied and saved: txz\u0043u004fu004eu0046u0045u0053u0053u002du0034u0037.jpg
Copied and saved: txz\u0046u0052u0049u0053u0054u004fu004eu002du0033.jpg
Copied and saved: txz\u004cu0061u006eu0063u0065u0074u002du0032.jpg
Copied and saved: txz\u004du006fu006eu0033u0074u0072.jpg
Copied and saved: txz\u0050u0068u006fu006eu006fu0052u002du0030.jpg
Copied and saved: txz\u0054u0048u0052u004du002du0045u0058.jpg
Copied and saved: txz\u0055u002du004fu0066u0066u0069u0063u0069u0061u006c.jpg
Copied and saved: txz\u0057.jpg
Copied and saved: txz\u4e07u9877.jpg
Copied and saved: txz\u4e34u5149.jpg
Copied and saved: txz\u4e4cu5c14u6bd4u5b89.jpg
Copied and saved: txz\u4e4cu6709.jpg
Copied and saved: txz\u4e91u8ff9.jpg
Copied and saved: txz\u4e9au53f6.jpg
Copied and saved: txz\u4ec7u767d.jpg
Copied and saved: txz\u4ee4.jpg
Copied and saved: txz\u4f0au5185u4e1d.jpg
Copied and saved: txz\u4f0au6851.jpg


In [26]:
from PIL import Image
import json
import os

def colors_are_similar(c1, c2, tolerance):
    return sum((a - b) ** 2 for a, b in zip(c1, c2)) ** 0.5 <= tolerance

def image_to_json_grid(image_path, grid_width, grid_height, tolerance, input_data):
    # Load image
    img = Image.open(image_path).convert('RGB')
    img_width, img_height = img.size

    # Compute cell size
    cell_w = img_width // grid_width
    cell_h = img_height // grid_height

    palette = []
    color_to_index = {}
    pixel_grid = []

    for y in range(grid_height):
        row = []
        for x in range(grid_width):
            # Crop the cell
            left = x * cell_w
            top = y * cell_h
            right = left + cell_w
            bottom = top + cell_h
            cell = img.crop((left, top, right, bottom))

            # Average color
            colors = cell.getdata()
            avg = tuple(sum(c[i] for c in colors) // len(colors) for i in range(3))

            # Check similarity
            matched_index = None
            for i, pc in enumerate(palette):
                if colors_are_similar(avg, pc, tolerance):
                    matched_index = i
                    break
                    
            if matched_index is None:
                matched_index = len(palette)
                palette.append(avg)

            row.append(matched_index)
        pixel_grid.append(row)

    # Convert palette to hex strings
    palette = ['#{:02x}{:02x}{:02x}'.format(*c) for c in palette]

    # combine results with input_json
    operator_name = filename[:-4]
    input_data[operator_name].update({
        "palette": palette,
        "pixels": pixel_grid
    })
    
    


In [31]:
# Input folder and output JSON file
src_folder = "txz"
input_json = "operators.json"
output_json = "operators_1.json"

with open(input_json, 'r', encoding='utf-8') as f:
    input_data = json.load(f)

for filename in os.listdir(src_folder):
    if not filename.lower().endswith(".jpg"):
        continue
        
    # Extract name and index from the filename
    src_path = os.path.join(src_folder, filename)

    try:
        # Convert image to JSON grid
        image_to_json_grid(src_path, 10, 20, 20, input_data)
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        
# Save combined results
with open(output_json, 'w', encoding='utf-8') as f:
    json.dump(input_data, f, indent=None, ensure_ascii=False)


In [32]:
import os
import json
from PIL import Image

# Input JSON file and output folder
input_json = "operators_1.json"
output_folder = "reconstructed_images"
os.makedirs(output_folder, exist_ok=True)

# Load JSON data
with open(input_json, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Reconstruct images for each operator
for operator_name, operator_data in data.items():
    try:
        # Get palette and pixel grid
        palette = [tuple(int(color[i:i+2], 16) for i in (1, 3, 5)) for color in operator_data["palette"]]
        pixel_grid = operator_data["pixels"]

        # Determine image size
        grid_height = len(pixel_grid)
        grid_width = len(pixel_grid[0])
        cell_size = 20  # Size of each cell in pixels
        img_width = grid_width * cell_size
        img_height = grid_height * cell_size

        # Create a new image
        img = Image.new("RGB", (img_width, img_height))

        # Draw the image using the pixel grid and palette
        for y, row in enumerate(pixel_grid):
            for x, color_index in enumerate(row):
                color = palette[color_index]
                for yy in range(y * cell_size, (y + 1) * cell_size):
                    for xx in range(x * cell_size, (x + 1) * cell_size):
                        img.putpixel((xx, yy), color)

        # Save the reconstructed image
        output_path = os.path.join(output_folder, f"{operator_name}.jpg")
        img.save(output_path, "JPEG")
        print(f"Reconstructed and saved: {output_path}")
    except Exception as e:
        print(f"Error reconstructing {operator_name}: {e}")

Reconstructed and saved: reconstructed_images\u0031u0032u0046.jpg
Reconstructed and saved: reconstructed_images\u0043u0061u0073u0074u006cu0065u002du0033.jpg
Reconstructed and saved: reconstructed_images\u0043u004fu004eu0046u0045u0053u0053u002du0034u0037.jpg
Reconstructed and saved: reconstructed_images\u0046u0052u0049u0053u0054u004fu004eu002du0033.jpg
Reconstructed and saved: reconstructed_images\u004cu0061u006eu0063u0065u0074u002du0032.jpg
Reconstructed and saved: reconstructed_images\u004du006fu006eu0033u0074u0072.jpg
Reconstructed and saved: reconstructed_images\u0050u0068u006fu006eu006fu0052u002du0030.jpg
Reconstructed and saved: reconstructed_images\u0054u0048u0052u004du002du0045u0058.jpg
Reconstructed and saved: reconstructed_images\u0055u002du004fu0066u0066u0069u0063u0069u0061u006c.jpg
Reconstructed and saved: reconstructed_images\u0057.jpg
Reconstructed and saved: reconstructed_images\u4e07u9877.jpg
Reconstructed and saved: reconstructed_images\u4e34u5149.jpg
Reconstructed and 